<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Patient Satisfaction Analytics using Large Language Models and Enteprise Vector Store
  <br>
       <img id="teradata-logo" src="images/TeradataLogo.png" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<hr>


<b style = 'font-size:16px;font-family:Arial;color:#00233C'>What is Patient Satisfaction?</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Patient satisfaction measures how happy a patient is with their healthcare experience, reflecting whether their expectations were met. It's a key indicator of quality and impacts various aspects of healthcare, including clinical outcomes, patient retention, and legal claims. Effective communication, efficient processes, and positive interactions all contribute to higher patient satisfaction.</p>

<b style = 'font-size:16px;font-family:Arial;color:#00233C'>Why is Patient Satisfaction so important?</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Patient satisfaction is crucial in healthcare because it's a key indicator of both the quality of care and the overall patient experience. Satisfied patients are more likely to adhere to treatment plans, have better health outcomes, and recommend their healthcare providers. Additionally, patient satisfaction can influence factors like patient retention, loyalty, and the financial well-being of healthcare organizations. And provider reimbursement in healthcare is increasingly linked to patient satisfaction through higher reimbursement rates.</p>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Challenges with operationalizing AI</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Healthcare organizations of all types are struggling with how to best leverage the promise and the power of new Large Language Model and other AI models, tools, and techniques.  Some of the top challenges include:</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Privacy and confidentiality</b>.  Many model services require users to send their data and/or prompts to third-party systems over the open internet. Furthermore, models have not been trained or augmented with any non-public data; and therefore may not even be able to provide any actionable results</li>
    <li><b>Actionable insights</b>.  AI tools provide unprecedented capabilities in extracting <b>meaning</b> from unstructured data (text, images, audio, etc.).  Insight alone isn't enough - organizations must contextualize this meaning with existing analytics and data</li>
    <li><b>Scale</b>.  Value increases with automation and scale.  Processing and analyzing millions or billions of records, providing analytics to users, applications, and customers in real-time are both critical requirements for deriving true business value from AI</li>
    </ul>

<b style = 'font-size:28px;font-family:Arial;color:#00233C'>Demonstrations</b>
<hr>

<table style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'>
    <tr><td style = 'vertical-align:top' width = '25%'><p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Patient Sentiment Analysis</b>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Connect and inspect call center transcripts stored in Vantage, object storage, or third-party systems</li>
                <br>      
                <li>Extract sentiment using native ClearScape Analytics functions accessing external LLMs</li>
                <br>
                <li>Analyze results using native analytics functions</li>
            </ol>
        </td><td><img src = 'images/Sentiment_Analysis_AWS.png' width = '100%'></td></tr>
    </table>
    <hr>
<table style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'>
    <tr><td style = 'vertical-align:top' width = '25%'><p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Semantic Search for Targeted Patient Outreach</b>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Connect and inspect call center transcripts stored in Vantage, object storage, or third-party systems</li>
                <br>      
                <li>Generate embeddings based on the semantic meaning of the call-center transcripts</li>
                <br>
                <li>Identify patients for targeted outreach using built-in analytics and RAG capabilities</li>
            </ol>
        </td><td><img src = 'images/Semantic_Search_AWS.png' width = '100%'></td></tr>
</table>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python Package Imports</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Standard practice to import required packages and libraries; execute this cell to import packages for Teradata automation as well as machine learning, analytics, utility, and data management packages.</p> 

In [ ]:
%pip install -r requirements.txt

In [ ]:
from teradataml import *
from teradatagenai import TeradataAI, TextAnalyticsAI, VSManager, VectorStore, VSApi
from teradatasqlalchemy.types import *
import pandas as pd
import csv, sys, os, warnings
from collections import OrderedDict
import ipywidgets as widgets
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import plotly.graph_objects as go

from IPython.display import clear_output , display as ipydisplay, Markdown
import matplotlib.pyplot as plt

# Set display options for dataframes, plots, and warnings
%matplotlib inline
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True

# load vars json
with open('vars.json', 'r') as f:
    session_vars = json.load(f)

# Database login information
host = session_vars['environment']['host']
username = session_vars['hierarchy']['users']['business_users'][1]['username']
password = session_vars['hierarchy']['users']['business_users'][1]['password']

# UES Authentication information
ues_url = session_vars['environment']['UES_URI']
configure.ues_url = ues_url
pat_token = session_vars['hierarchy']['users']['business_users'][1]['pat_token']
pem_file = session_vars['hierarchy']['users']['business_users'][1]['key_file']


compute_group = session_vars['hierarchy']['users']['business_users'][1]['compute_group']

access_key = input('AWS Access Key: ')
secret_key = input('AWS Secret Key: ')

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Connect to the database</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a database connection and authenticate to the environment services for model management.</p> 

In [ ]:
# connect to database
eng = create_context(host = host, username = username, password = password)
#strip off the trailing /open-analytics
base_url = ues_url[:-15]

set_auth_token(base_url=base_url, pat_token=pat_token, pem_file=pem_file);


<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Sentiment Analysis
  <br>
    </p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Extract the emotion (anger, joy, sadness, love) and strength of that emotion (0-1) based on analysis of unstructured call center transcripts.  Inspect the distribution of how strongly patients are feeling these emotions - are we reaching a point where we need to change business practices or improve outreach?</p>

<table style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'>
    <tr><td style = 'vertical-align:top' width = '25%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Connect and inspect call center transcripts stored in Vantage, object storage, or third-party systems</li>
                <br>      
                <li>Extract sentiment using native ClearScape Analytics functions accessing external LLMs</li>
                <br>
                <li>Analyze results using native analytics functions</li>
            </ol>
        </td><td><img src = 'images/Sentiment_Analysis_AWS.png' width = '100%'></td></tr>
    </table>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Inspect the Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We create a special type of DataFrame which behaves normally yet represents large, interconnected data sets at scale. The Dataframe can be displayed pandas-style, or we can enable an interactive data and function browser.  This intereactive widget leverages <b>in-database</b> analytic and exploratory functions, allowing users to interact with the data at any scale.</p>


In [ ]:
tdf_transcripts = DataFrame('"demo_ofs"."patient_feedback"')
tdf_transcripts

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Set up the Sentiment Extraction model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <b>teradatagenai</b> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local GPU compute. In this case we will download and deploy the <i><a href = 'https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion'>bhadresh-savani/distilbert-base-uncased-emotion</a></i> for sentiment extraction tasks.</p>


In [ ]:
# Provide model details
model_name = 'anthropic.claude-instant-v1'

# Select in-database or external model
llm = TeradataAI(api_type = 'AWS',
         model_name = model_name,
         region = 'us-west-2',
         # authorization = 'Repositories.BedrockAuth'
         access_key = access_key,
         secret_key = secret_key)


obj = TextAnalyticsAI(llm=llm)

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Extract Patient Sentiment</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A simple method call will extract the sentiment for patient comments <b>in-database</b> using the desired LLM and CSP provider.</p>

In [ ]:
display.print_sqlmr_query = False
tdf_sentiment = obj.analyze_sentiment(column='Transcript', data=tdf_transcripts)
tdf_sentiment

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Execute simple descriptive analytics</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now, we can begin asking increasingly more complex questions of the data; what's the distribution of sentiment?  Or - does any individual patient have increasing feelings of anger, fear, or sadness?  Native, in-database <b>ClearScape Analytics</b> functions can answer these questions at extreme speed and scale.</p>

In [ ]:
CategoricalSummary(data = tdf_sentiment, target_columns = 'Sentiment').result

In [ ]:
df = CategoricalSummary(data = tdf_sentiment, target_columns = 'Sentiment').result.to_pandas()
fig = px.bar(df, x = 'DistinctValue', y = 'DistinctValueCount')
fig.show()

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Develop targeted patient outreach with Semantic Search
  <br>
    </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Match our patients with <b>targeted</b> outreach programs based on the <b>Semantic Meaning</b> of their comments history.  This can allow our organization to provide tailored programs or assistance based on the intent of the commentary, allowing for more accurate targeting.</p>

<table style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'>
    <tr><td style = 'vertical-align:top' width = '25%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Create a Vector Store using CSP LLM services and store data in Vantage</li>
                <br>      
                <li>Perform interactive analysis and generate recommended responses to customer complaints</li>
                <br>
                <li>Identify patients for targeted outreach using similarity search functions</li>
            </ol>
        </td><td><img src = 'images/Semantic_Search_AWS.png' width = '100%'></td></tr>
</table>
<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Enterprise Vector Store</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Vector Embedding</b> is a numerical representation of data that captures semantic relationships and similarities, making it possible to perform mathematical operations and comparisons on the data for various tasks like text analysis and recommendation systems.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Teradata AI Factory</b> makes this process simple by leveraging highly-scalable, secure, <b>On-Premises</b> GPU resources via NVIDIA NIM architecture using simple, developer-friendly interfaces.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Generative Responses</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use Natural Language</b> to inspect the patient complaints data, and leverage <b>Private LLMs</b> to generate rich, actionable responses.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Hyper-personalize</b> the appropriate action based on AI-based semantic analysis and search.</p>

In [ ]:
vs = VectorStore(name = 'Transcript_Analysis_test')

In [ ]:
# if desired, destroy and recreate the vector store - or to use an existing one
# skip to the "similarity_search" code below
# vs.destroy()

In [ ]:
# Python
vs.create(search_algorithm = 'VECTORDISTANCE',
          object_names = DataFrame('"demo_ofs"."patient_feedback_no_dup"'),
          key_columns = ['Call_ID', 'first_name', 'last_name'],
          data_columns = ['Transcript'],
          vector_column = 'Embedding')

In [ ]:
vs.status()

In [ ]:
vs.similarity_search(question = 'top 10 most angry unique callers').similar_objects[['score','Call_ID','Transcript']]

In [ ]:
ipydisplay(Markdown(vs.ask(question = 'the top angry customers', prompt = 'suggest a targeted outreach program')));

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Construct a list of patient care programs</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a small list of topics that will represent the outreach programs we want to target.  This could represent existing outreach or marketing programs we want to more accurately target.

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Sample patient care programs</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Late test results</b></li>
    <li><b>Long wait times</b></li>
    <li><b>Missing prescriptions</b></li>
    <li><b>Argumentative staff</b></li>
    <li><b>Unhelpful advice</b></li>
    </ul>

In [ ]:
l = ['Late test results',
      'Long wait times',
      'Missing prescriptions',
      'Argumentative staff',
      'Unhelpful advice'
      ]

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Identify patient and program match</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Provide these results in real-time to call center agent dashboards, content management systems, or other marketing automation applications.</p>

In [ ]:
topics_list = l

d = widgets.Dropdown(options=topics_list)

output = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output()
            local_df = vs.similarity_search(question = change['new']).similar_objects[['first_name', 'last_name','Transcript']].to_pandas()
            ipydisplay(local_df)
            text = " ".join(comment for comment in local_df['Transcript'])
            stopwords = set(STOPWORDS)
            wordcloud = WordCloud(stopwords = stopwords, width = 800, height=400, 
                                  max_words=100, min_word_length=1, 
                                  collocations = True, background_color = 'white').generate(text)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud, interpolation='gaussian')
            plt.axis('off')
            plt.title(f'Word Cloud for Search Topic')
            plt.show()
            
d.observe(on_change)

print('Select Topic to Search:')
ipydisplay(d, output)


<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Conclusion - Operationalizing AI-powered analytics</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The preceding demo showed two real-world applications of AI-powered analytics that leverages <b>private</b> data <b>securely</b>, to allow for deeper insight into unstructured data that can improve patient satisfaction.</p>

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Cleanup</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Call the destroy() method of the VS object to clean up the objects created during this demo.</p>

In [ ]:
vs.destroy()

In [ ]:
remove_context()